In [6]:
from landnet.modelling.segmentation.models import DeepLabV3ResNet50Builder
from landnet.features.grids import get_grid_for_variable
from landnet.enums import GeomorphometricalVariable, Mode
from landnet.features.tiles import TileConfig, TileSize
from landnet.modelling.segmentation.lightning import (
    LandslideImageSegmenter,
    LandslideImageSegmentationDataModule,
)
from landnet.modelling import torch_clear
import lightning as L
from torch.utils.data import random_split
from landnet._typing import TuneSpace

torch_clear()

In [7]:
variables = [
    GeomorphometricalVariable.SLOPE,
    GeomorphometricalVariable.REAL_SURFACE_AREA,
    GeomorphometricalVariable.PROFILE_CURVATURE,
    GeomorphometricalVariable.DIGITAL_ELEVATION_MODEL,
    GeomorphometricalVariable.GENERAL_CURVATURE,
    GeomorphometricalVariable.HILLSHADE,
    GeomorphometricalVariable.TOPOGRAPHIC_POSITION_INDEX,
    GeomorphometricalVariable.NEGATIVE_TOPOGRAPHIC_OPENNESS,
]
tile_config = TileConfig(TileSize(100, 100), overlap=0)
model_config: TuneSpace = {
    'batch_size': 4,
    'learning_rate': 0.0001,
    'tile_config': tile_config,
}

In [8]:
model = DeepLabV3ResNet50Builder(2).build(
    in_channels=len(variables), mode=Mode.TRAIN
)
dm = LandslideImageSegmentationDataModule(model_config, variables)
segmenter = LandslideImageSegmenter(model_config, model, 2)
trainer = L.Trainer(
    limit_train_batches=100, max_epochs=10, enable_checkpointing=False
)

INFO: Set weights to DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1


INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


In [9]:
model

Sequential(
  (0): Conv2d(8, 3, kernel_size=(1, 1), stride=(1, 1))
  (1): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1

In [10]:
model = trainer.fit(model=segmenter, datamodule=dm)

INFO: Setting up train, test and validation datasets based on [<GeomorphometricalVariable.SLOPE: 'slope'>, <GeomorphometricalVariable.REAL_SURFACE_AREA: 'area'>, <GeomorphometricalVariable.PROFILE_CURVATURE: 'cprof'>, <GeomorphometricalVariable.DIGITAL_ELEVATION_MODEL: 'dem'>, <GeomorphometricalVariable.GENERAL_CURVATURE: 'cgene'>, <GeomorphometricalVariable.HILLSHADE: 'shade'>, <GeomorphometricalVariable.TOPOGRAPHIC_POSITION_INDEX: 'tpi'>, <GeomorphometricalVariable.NEGATIVE_TOPOGRAPHIC_OPENNESS: 'nego'>]


INFO: Finished setting up train and validation datasets.


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name            | Type                         | Params | Mode 
-------------------------------------------------------------------------
0 | model           | Sequential                   | 39.6 M | train
1 | criterion       | CrossEntropyLoss             | 0      | train
2 | train_metrics   | SegmentationMetricCollection | 0      | train
3 | val_metrics     | SegmentationMetricCollection | 0      | train
4 | test_metrics    | SegmentationMetricCollection | 0      | train
5 | predict_metrics | SegmentationMetricCollection | 0      | train
-------------------------------------------------------------------------
39.6 M    Trainable params
0         Non-trainable params
39.6 M    Total params
158.536   Total estimated model params size (MB)
202       Modules in train mode
0         Modules in eval mode


Epoch 0:   0%|          | 0/1 [01:16<?, ?it/s]                             


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 256, 1, 1])

In [ ]:
trainer.test(model=segmenter, dataloaders=dm)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 100/100 [00:45<00:00,  2.21it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           Test metric                     DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_dice_score_epoch             0.9098041653633118
test_generalized_dice_score_epoch       0.7022680640220642
         test_loss_epoch                0.47157660126686096
         test_mIoU_epoch                0.4549010992050171
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_dice_score_epoch': 0.9098041653633118,
  'test_generalized_dice_score_epoch': 0.7022680640220642,
  'test_mIoU_epoch': 0.4549010992050171,
  'test_loss_epoch': 0.47157660126686096}]